In [3]:
import pickle, fileinput
import json

In [4]:
import sys, os


In [49]:
from PIL import Image, ImageDraw
from collections import defaultdict

def extract_commands(file):
    pre_parts = file.split('[command]')
    parts = [pre_part.split('[/command]')[0] for pre_part in pre_parts if '[/command]' in pre_part]
    return parts

def count_moves_heat(command, cache, mode='all'):
    if '[move]' not in command:
        return []
    x_co = command.split('x=\"')[1].split('\"\n')[0].split(',')
    y_co = command.split('y=\"')[1].split('\"\n')[0].split(',')
    if mode == 'all':
        return list(zip(x_co, y_co))[1:]
    if mode == 'finish':
        return [[x_co[0], y_co[0]], [x_co[-1], y_co[-1]]]

def count_recruit_heat(command, cache):
    if '[recruit]' not in command:
        return []
    x_co = command.split('x=')[1].split('\n')[0]
    y_co = command.split('y=')[1].split('\n')[0]
    return [[x_co, y_co]]

def count_fight_heat(command, cache):
    if '[attack]' not in command:
        return []
    x_at = command.split('x=')[1].split('\n')[0].split(',')
    y_at = command.split('y=')[1].split('\n')[0].split(',')
    x_def = command.split('x=')[2].split('\n')[0].split(',')
    y_def = command.split('y=')[2].split('\n')[0].split(',')
    return [[x_at[0], y_at[0]], [x_def[0], y_def[0]]]

def count_attack_heat(command, cache):
    heat = count_fight_heat(command)
    if heat:
        return [heat[0]]
    return []

def count_defence_heat(command, cache):
    heat = count_fight_heat(command)
    if heat:
        return [heat[1]]
    return []

def count_mv_rc_heat(command, cache):
    return count_moves_heat(command, cache) + count_recruit_heat(command, cache)

def count_color(hot, max_hot, mult = 1):
    hott = min(int(255 * mult * hot / max_hot), 255)
    if hott == 0:
        return (0, 0, 0)
    return (hott, 0, 255 - hott)

def to_hex(i, j):
    return str(i) + "_" + str(j)

def no_cache(command, cache):
    return {}

def generate_single_heat_map(dir_path, files, map_path, new_map_name, 
                        count_heat_by_command,
                        hexes_width, hexes_hight, multiplier,
                        mark_size, pixel_shift_odd, pixel_shift_even, main_shift,
                        alpha=0.65, show=False,  players_mod='both', count_cache_by_command=no_cache
):
    warm_map = defaultdict(int)

    for i in range(1, hexes_width):
        for j in range(1, hexes_hight):
            warm_map[str(i) + '_' + str(j)] = 0
    cache = {}
    for file in files:
        filename = dir_path + file
        with open(filename, 'r') as f:
            text = f.read()
        steps = extract_steps(text)
        for command in commands:
            if players_mod != 'both':
                if 'from_side=' + str(players_mod) not in step:
                    continue
            cache = count_cache_by_command(command, cache)
            heat_hexes = count_heat_by_command(command, cache)
            for hexx in heat_hexes:
                warm_map[to_hex(hexx[0], hexx[1])] += 1
            
    image = Image.open(map_path)
    image_copy = image.copy()

    max_hot = max(warm_map.values())
    for i in range(1, hexes_width, 2):
        for j in range(1, hexes_hight):
            canvas = Image.new('RGB', (mark_size, mark_size),
                               count_color(warm_map[to_hex(i, j)], max_hot, multiplier))
            image_copy.paste(canvas, (pixel_shift_odd[0] + (i - 1) * main_shift[0],
                                      pixel_shift_odd[1] + (j - 1) * main_shift[1]))

    for i in range(2, hexes_width, 2):
        for j in range(1, hexes_hight):
            canvas = Image.new('RGB', (mark_size, mark_size),
                               count_color(warm_map[to_hex(i, j)], max_hot, multiplier))
            image_copy.paste(canvas, (pixel_shift_even[0] + (i - 1) * main_shift[0],
                                      pixel_shift_even[1] + (j - 1) * main_shift[1]))
    
    image = Image.blend(image, image_copy, alpha=alpha)
    if new_map_name:
        image_copy.save(new_map_name)
    if show:
        image_copy.show()

In [50]:
def generate_all_maps(dir_path, files, map_path, new_map_name,
                        hexes_width, hexes_hight, 
                        mark_size, pixel_shift_odd, pixel_shift_even, main_shift,
                      alpha=0.65, show=False
):
    generate_single_heat_map(dir_path=dir_path, files=files, map_path=map_path, new_map_name=new_map_name + '_mv_rc_2.png',
                             count_heat_by_command=count_mv_rc_heat,
                             hexes_width=hexes_width, hexes_hight=hexes_hight, multiplier=2,
                             mark_size=mark_size,
                             pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift,
                             alpha=0.65, show=show, players_mod='both')


In [51]:
dir_path = 'replays_wl/'
files = ["770", "775", "781", "782", "783", "784", "869", "875", "895", "913", "922", "925", "1478"]
map_path = 'winter_steps.png'

main_forward_shift = 27 # cool
main_down_shift = 36 # cool
main_shift =  (27, 36)
pixel_shift_odd = (10, 32)
pixel_shift_even = (10, 48)


In [52]:
generate_all_maps(dir_path=dir_path, files=files, map_path=map_path, new_map_name='ws', show=True,
                        hexes_width=44, hexes_hight=40,
                        mark_size=18, pixel_shift_odd=pixel_shift_odd, pixel_shift_even=pixel_shift_even, main_shift=main_shift, alpha=0.65
)